In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
from torch import device, cuda

import pandas as pd

In [2]:
reports_file_path = './data/br/netbeans/netbeans.csv'
relations_file_path = 'data/br/netbeans/netbeans_pairs - Copia.csv'

reports = pd.read_csv(reports_file_path, index_col='bug_id')
relations = pd.read_csv(relations_file_path)

In [43]:
added_bug_reports = set()
train_examples = []
class_label = 0

invalid_ids = set()

for index, row in relations.iterrows():
    bug_id = int(row[0])
    bug = None
    try:
        bug = reports.iloc[bug_id]
    except:
        invalid_ids.add(bug_id)

    if ( not pd.isna(row[1]) and bug_id not in added_bug_reports and bug is not None):
        duplicates_ids = [int(x) for x in row[1].split(";")]
        
        added_bug_reports_len_before = len(added_bug_reports)

        for duplicate_id in duplicates_ids:
            duplicate = None
            try:
                duplicate = reports.iloc[duplicate_id]
            except:
                invalid_ids.add(duplicate_id)

            if duplicate is not None and duplicate_id not in added_bug_reports:
                train_examples.append(
                    InputExample(
                        texts=[duplicate['description']],
                        label=class_label
                    )
                )
                added_bug_reports.add(duplicate_id)
        
        # The loss function we are using requires that there are at leas 2 examples from each label,
        # therefore we only add the "main bug" if at leats one of its duplicates was added
        if len(added_bug_reports) > added_bug_reports_len_before:
            train_examples.append(
                InputExample(
                    texts=[bug['description']],
                    label=class_label
                )
            )
            added_bug_reports.add(bug_id)

            class_label +=1

print(len(added_bug_reports))
print(len(invalid_ids))

C:\Users\lpros\AppData\Local\Temp\ipykernel_7812\1233618834.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bug_id = int(row[0])
C:\Users\lpros\AppData\Local\Temp\ipykernel_7812\1233618834.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ( not pd.isna(row[1]) and bug_id not in added_bug_reports and bug is not None):
C:\Users\lpros\AppData\Local\Temp\ipykernel_7812\1233618834.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[po

46321
10248


In [44]:
print(len(train_examples))

46321


In [45]:
print(class_label)

18123


In [46]:
print(len(reports))

216715
